In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset,DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from datetime import datetime
import pandas_profiling as ppf
# from tensorboardX import SummaryWriter
# writer=SummaryWriter(logdir='./neuron')

In [5]:
encoded_all_data=pd.read_csv(r'./data/created_data/encoded_all_data.csv').reset_index(drop=True).set_index('ID')
encoded_all_data

,flag,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge
ID,,,,,,,,,,,,,
28.0,1.0,-0.376463,-0.376167,-0.226258,-0.226812,-0.391587,-0.247471,-0.004741,-0.463019,0.406379,0.405600,-0.338439,-0.399303
230.0,1.0,-0.376463,-0.376167,-0.131425,-0.132086,-0.305196,-0.402976,0.106861,-0.463019,-0.232299,-0.232355,-0.547484,-0.371525
429.0,1.0,0.422401,0.422863,-0.325154,-0.325293,-1.181800,0.637125,-0.461944,-0.463019,1.593755,1.592084,1.565990,1.380232
693.0,1.0,-0.376463,-0.376167,-0.096611,-0.097367,-0.583686,0.199842,0.356249,-0.463019,0.278852,0.278391,0.119368,-0.410640
727.0,1.0,-0.376463,-0.376167,0.276225,0.274462,-1.224502,0.836748,-1.434351,-0.463019,2.129373,2.129724,1.686916,2.114191
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995.0,0.0,2.477889,2.478143,-0.325154,-0.325293,0.220857,1.231584,1.261654,1.235796,0.248468,0.247847,0.656345,0.435555
5999996.0,0.0,-0.254723,-0.254376,-0.325154,-0.325293,-0.121522,-0.581308,0.239991,-0.356314,-0.176726,-0.176689,-0.696066,-0.730380
5999997.0,0.0,0.126307,0.126595,0.714218,0.710454,0.567215,0.303538,0.254127,0.852716,-0.285456,-0.285470,0.119244,-0.294231


In [3]:
# making all variables categorical
for col in encoded_all_data.columns:
    encoded_all_data[col] = encoded_all_data[col].astype('category')

In [6]:
# txt=encoded_all_data[['行业','企业类型','控制人类型','区域']]
# txt

In [ ]:
# #categorical embedding for columns having more than two values?
# embedded_cols = {n: len(col.cat.categories) for n,col in txt.items()}
# embedded_cols

In [ ]:
# embedded_col_names = embedded_cols.keys()
# print(embedded_col_names)
# len(encoded_all_data.columns) - len(embedded_cols) #number of numerical columns

In [ ]:
# embedding_sizes = [(n_categories, min(50, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]
# embedding_sizes

In [7]:
init_x=encoded_all_data.dropna(subset=['flag']).drop(['flag'],axis=1)
init_y=encoded_all_data.dropna(subset=['flag'])['flag'].to_numpy()
train_X,val_X, train_y, val_y = train_test_split(init_x,init_y, test_size=0.30, random_state=0)
display(train_X,train_y)

,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge
ID,,,,,,,,,,,,
5992282.0,-0.140242,-0.139879,-0.325154,-0.325293,-0.114495,-0.612358,0.269269,-0.340802,-0.285456,-0.285470,-0.693239,-0.705462
5982045.0,1.966176,1.966437,0.052762,-0.216089,1.596196,1.148906,0.533604,3.938261,-0.285456,-0.285470,2.712321,1.191353
895831.0,0.786722,0.787260,-0.325154,-0.325293,-0.716335,0.826067,0.255958,-0.463019,0.547068,0.546235,0.432168,0.348116
5985243.0,-0.322979,-0.322662,-0.325154,-0.214615,-0.155016,-0.640345,0.219881,-0.380627,-0.283383,-0.283391,-0.729194,-0.748672
5983883.0,-0.042745,-0.042454,-0.218925,-0.219351,-0.306733,-0.338618,0.228194,-0.386355,-0.285456,-0.190409,-0.401226,-0.508057
...,...,...,...,...,...,...,...,...,...,...,...,...
5984896.0,1.196130,1.196813,-0.325154,-0.325293,1.549562,0.811447,0.434845,1.086850,-0.285456,-0.285470,0.414652,0.372022
5996407.0,-0.376463,-0.376167,0.036678,0.037324,0.111706,-0.215827,0.511119,-0.046966,-0.062900,-0.062541,-0.365487,-0.666507
5994688.0,-0.376463,-0.376167,-0.132856,-0.133514,0.123423,-0.215391,-0.447947,0.100624,0.266530,0.267311,-0.224783,0.548445


array([0., 0., 1., ..., 0., 0., 1.])

In [50]:
class CompanyDataset(Dataset):
    def __init__(self, X, Y):#, embedded_col_names
        X = X.copy()
#         self.X1 = X.loc[:,embedded_col_names].copy().values.astype(np.int64) #categorical columnss
#         self.X2 = X.drop(columns=embedded_col_names).copy().values.astype(np.float32) #numerical columns
        self.x = X.copy().values.astype(np.float32)
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
#         return self.X1[idx], self.X2[idx], self.y[idx]
        return self.x[idx],self.y[idx]

In [51]:
#creating train and valid datasets
train_ds = CompanyDataset(train_X, train_y,)#embedded_col_names
valid_ds = CompanyDataset(val_X,val_y,)#embedded_col_names

In [52]:
# train_X
train_y

array([0., 0., 1., ..., 0., 0., 1.])

In [53]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [54]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [55]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [56]:
device = get_default_device()
device

device(type='cuda')

In [57]:
class FlagModel(nn.Module):  #分为两部分categorical 和 continuous
    def __init__(self,n):#embedding_sizes, n_cont
        super().__init__()
#         self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
#         n_emb = sum(e.embedding_dim for e in self.embeddings) #length of all embeddings combined  
        
#         self.n_emb, self.n_cont = n_emb, n_cont
        
        self.lin1 = nn.Linear(n, 200)  #线性层输入： 输出：(200)  self.n_emb + self.n_cont
        self.lin2 = nn.Linear(200, 70)
        self.lin3 = nn.Linear(70, 2)
#         self.bn1 = nn.BatchNorm1d(self.n_cont)
        self.bn2 = nn.BatchNorm1d(200)
        self.bn3 = nn.BatchNorm1d(70)
        
#         self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
        

    def forward(self,x):  #描述了一个前向计算图 x_cat, x_cont
#         x = [e(x_cat[:,i]) for i,e in enumerate(self.embeddings)]   #(embeddings): ModuleList(
#                                                                                  (0): Embedding(992, 50)
#                                                                                  (1): Embedding(51, 26)
#   )
#         x = torch.cat(x, 1)
#         x = self.emb_drop(x)  #把categorical  drop_out
        
#         x2 = self.bn1(x_cont)   #把continuous标准化
        
#         x = torch.cat([x, x2], 1)  #把categorical 和 continuous 合并
        x = F.relu(self.lin1(x))  #线性层并激活
        x = self.drops(x)        #drop_out
        
        x = self.bn2(x)         #标准化
        x = F.relu(self.lin2(x)) #线性层并激活
        x = self.drops(x)        #drop_out
        x = self.bn3(x)          #标准化
        x = self.lin3(x)         #线性层

#         x=torch.sigmoid(x)
        return x

In [58]:
model = FlagModel(12)
to_device(model, device)

FlagModel(
  (lin1): Linear(in_features=12, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=70, bias=True)
  (lin3): Linear(in_features=70, out_features=2, bias=True)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drops): Dropout(p=0.3, inplace=False)
)

In [59]:
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters()) #filter过滤序列，留下True
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [60]:
def train_model(model, optim, train_dl,i):
    model.train()
    total = 0
    sum_loss = 0
    for x1, y in train_dl:
        batch = y.shape[0]
        output = model(x1)
        y=y.long()
        loss = F.cross_entropy(output, y)   
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
#     writer.add_scalar('train_loss',sum_loss/total,i)
    return sum_loss/total

In [61]:
def val_loss(model, valid_dl,i):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x1,y in valid_dl:
        current_batch_size = y.shape[0]
        out = model(x1)
        y=y.long()
        loss = F.cross_entropy(out, y)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        pred = torch.max(out, 1)[1]
        correct += (pred == y).float().sum().item()
#         correct = torch.mean((pred == y).float())
#     writer.add_scalar('valid_loss',sum_loss/total,i)
#     writer.add_scalar('valid_acc',correct/total,i)
    print("valid loss %f and accuracy %f " % (sum_loss/total, correct/total))
    return sum_loss/total, correct/total

In [62]:
def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs): 
        loss = train_model(model, optim, train_dl,i)
        print("training loss: ", loss)
        _,acc=val_loss(model, valid_dl,i)
        if acc>0.99:
            break
    return 

In [63]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size,shuffle=True)

In [64]:
train_dl = DeviceDataLoader(train_dl,device)
valid_dl = DeviceDataLoader(valid_dl,device)

In [65]:
train_loop(model, epochs=1000, lr=0.0001, wd=0.05)

training loss:  0.7163296199602598
valid loss 0.641088 and accuracy 0.613838 
training loss:  0.5966178901911269
valid loss 0.525381 and accuracy 0.769799 
training loss:  0.5200230606234404
valid loss 0.451182 and accuracy 0.835531 
training loss:  0.4632277373526515
valid loss 0.397299 and accuracy 0.870781 
training loss:  0.42095123046587135
valid loss 0.352325 and accuracy 0.895839 
training loss:  0.3824972685762232
valid loss 0.312351 and accuracy 0.919776 
training loss:  0.3512538374616871
valid loss 0.283629 and accuracy 0.927817 
training loss:  0.32736037467155016
valid loss 0.258092 and accuracy 0.934268 
training loss:  0.3067403861082604
valid loss 0.237258 and accuracy 0.938943 
training loss:  0.2891042162453546
valid loss 0.221890 and accuracy 0.941655 
training loss:  0.273335045095255
valid loss 0.210387 and accuracy 0.942964 
training loss:  0.2561964112141851
valid loss 0.196228 and accuracy 0.947359 
training loss:  0.24681896427774297
valid loss 0.185443 and acc

training loss:  0.08316412713950905
valid loss 0.081388 and accuracy 0.971669 
training loss:  0.0819436456974234
valid loss 0.074537 and accuracy 0.976344 
training loss:  0.08009893667157826
valid loss 0.074442 and accuracy 0.977653 
training loss:  0.08042078757836184
valid loss 0.073654 and accuracy 0.977186 
training loss:  0.08183997974055891
valid loss 0.072829 and accuracy 0.977934 
training loss:  0.0796487650102529
valid loss 0.072491 and accuracy 0.977747 
training loss:  0.07833703323008565
valid loss 0.072204 and accuracy 0.978214 
training loss:  0.08103154991767843
valid loss 0.074219 and accuracy 0.976344 
training loss:  0.07932913632505008
valid loss 0.070666 and accuracy 0.978121 
training loss:  0.07814240558907842
valid loss 0.072799 and accuracy 0.977092 
training loss:  0.0758460185586133
valid loss 0.070387 and accuracy 0.978588 
training loss:  0.07581838355138966
valid loss 0.071561 and accuracy 0.977092 
training loss:  0.0761633147612266
valid loss 0.069457 

### 以下为贴标签部分

In [79]:
# 目前最好：  99.0276  lr=0.0001 dw=0.05 adam
torch.save(model.state_dict(), "optim_old.npy")
model.load_state_dict(torch.load("optim_old.npy"))

In [27]:
# writer.close()

In [66]:
non_label=encoded_all_data.drop(init_x.index)
non_label_X=non_label.drop(['flag'],axis=1)
estimate_y=np.array([1]*non_label_X.shape[0])
non_label_X

,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge
ID,,,,,,,,,,,,
1080523.0,0.153910,0.154316,0.334840,0.335179,-0.423658,-0.235871,-0.522061,-0.463019,-0.285456,-0.285470,-0.258610,0.416919
1080756.0,2.037227,2.037933,-0.188487,-0.189200,-0.737513,1.150920,-0.921308,-0.463019,-0.285456,-0.285470,0.495737,1.237598
1080951.0,-0.376463,-0.376167,0.396778,0.394059,-0.603652,0.351831,0.076741,-0.463019,-0.285456,-0.285470,0.175824,0.557640
1080972.0,-0.376463,-0.376167,-0.254944,-0.255188,-0.230101,-0.618374,0.206208,-0.463019,-0.263837,-0.263875,-0.726563,-0.773093
1081027.0,-0.376463,-0.376167,-0.325154,-0.325293,-0.415439,-0.323241,0.438912,-0.463019,0.185092,0.184636,-0.283236,-0.388421
...,...,...,...,...,...,...,...,...,...,...,...,...
5999614.0,1.952389,1.953307,-0.325154,-0.325293,0.250238,0.206426,0.982232,0.157481,-0.285456,-0.285470,0.070694,-0.035223
5999746.0,-0.376463,-0.376167,0.100278,0.098578,0.038375,-0.239628,-0.104054,0.203587,-0.042700,-0.042990,-0.275084,-0.262263
5999945.0,-0.376463,-0.376167,-0.302229,-0.213973,-0.191376,-0.644977,0.255412,-0.449279,-0.276388,-0.276409,-0.761829,-0.810960


In [67]:
# making all variables categorical
for col in non_label.columns:
    non_label[col] = non_label[col].astype('category')

In [68]:
# #categorical embedding for columns having more than two values
# embedded_cols = {n: len(col.cat.categories) for n,col in txt.items()}
# embedded_cols

In [69]:
# embedding_sizes = [(n_categories, min(50, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]
# embedding_sizes

In [70]:
test_ds = CompanyDataset(non_label_X,estimate_y) #,embedded_col_names
batch_size2 = non_label_X.shape[0]
test_dl = DataLoader(test_ds,batch_size=batch_size2,shuffle=False) #返回的是可迭代对象
# first=iter(test_dl)
# next(first)
test_dl = DeviceDataLoader(test_dl, device)

In [71]:
def predict(model, test_dl):
    model.eval()
    for x1, y in test_dl:
        out = model(x1)
        pred = torch.max(out, 1)[1]
        return pred.tolist()

In [72]:
flag=predict(model,test_dl)
flag

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [75]:
# neuron_final=pd.concat([pd.DataFrame(index=non_label_X.index,data=flag,columns=['flag']),init_x],axis=1)
# neuron_final=encoded_all_data.combine_first(neuron_final)
neuron_final.to_csv(r'./data/created_data/neuron_final.csv')
# neuron_final.info()

In [77]:
# prf=ppf.ProfileReport(neuron_final)
prf.to_file('report.html')